In [173]:
#def my_likelihood():
#    return


In [174]:
##Priors Info

# k1 = uniform(1.7*10**-9,1.06*10**-9)##...?
# GA1 = uniform(1.4,3.6)
# GA2 = uniform(0,8)
# GA3 = uniform(0.5,7.5)
# d1 = uniform(229.44,954.72)
#d2 = uniform(d1 + 487.610 , 1652.93)

In [175]:
#Conversion for scaling

MeV = 1 
GeV = 1000*MeV
fm  = 1
c   = 1
metres = (10**15)*fm
km = 1000*metres
second = 2.99*(10**8)*metres
kg = (second**2)/((1.6*(10**(-13)))*((metres)**2))
G = 6.67*10**(-11)*((metres)**3)/(kg*(second)**2)
mbar = 1.66*(10**-27)*kg

import math
dp = 10**-12
mbar = 1.66*(10**-27)*kg
pi = math.pi
Rs = ((2*G*(1.98847*(10**30)*kg))/((c**2)))
K1 =  ((4*pi*(Rs**3))/(1.98847*((10**30)*kg)*(c**2)))*(GeV/(fm)**3)

#importing libraries
import numpy as np
import scipy
import matplotlib.pyplot as plot
from scipy.integrate import odeint
from scipy.stats import uniform

#SFHx
class TOV_Solver:
    def __init__(self,Pc,p1,p2,k1,GA1,GA2,GA3,GA1_R,GA2_R,GA3_R,GA4_R,k1_R,
                 k2_R,k3_R,k4_R,P1_Read,P2_Read,P3_Read):
        self.Pc = Pc
        self.p1 = p1
        self.p2 = p2
        self.k1 = k1
        self.GA1 = GA1
        self.GA2 = GA2
        self.GA3 = GA3
        self.GA1_R = GA1_R
        self.GA2_R = GA2_R
        self.GA3_R = GA3_R
        self.GA4_R = GA4_R
        self.k1_R = k1_R
        self.k2_R = k2_R
        self.k3_R = k3_R
        self.k4_R = k4_R
        self.P1_Read = P1_Read
        self.P2_Read = P2_Read
        self.P3_Read = P3_Read
    
    def piecewise_polytrope_SFHx(self,p):
    
        k2 = self.p1/((self.p1/self.k1)**(1/self.GA1) + self.p1/(self.GA1-1) - self.p1/(self.GA2-1))**self.GA2
        k3 = self.p2/((self.p2/k2)**(1/self.GA2) + self.p2/(self.GA2-1) - self.p2/(self.GA3-1))**self.GA3
        
        condition1_Read = p < P1_Read
        condition2_Read = (p >= P1_Read) & (p < P2_Read)
        condition3_Read = (p >= P2_Read) & (p < P3_Read)
                
        condition1 = (p >= P3_Read) & (p < self.p1)
        condition2 = (p >= self.p1) & (p < self.p2)
        condition3 = (p >= self.p2) 
    
        condition = [condition1_Read,condition2_Read,condition3_Read,
                    condition1, condition2, condition3]
        Polytrope1 = lambda p: ((GeV*p/self.k1)**(1/self.GA1) + (p*GeV/(self.GA1-1)))/GeV
        Polytrope2 = lambda p: ((GeV*p/k2)**(1/self.GA2) + (p*GeV/(self.GA2-1)))/GeV
        Polytrope3 = lambda p: ((GeV*p/k3)**(1/self.GA3) + (p*GeV/(self.GA3-1)))/GeV
        
        Polytrope1_Read = lambda p: ((GeV*p/self.k1_R)**(1/self.GA1_R) + (p*GeV/(self.GA1_R-1)))/GeV
        Polytrope2_Read = lambda p: ((GeV*p/self.k2_R)**(1/self.GA2_R) + (p*GeV/(self.GA2_R-1)))/GeV
        Polytrope3_Read = lambda p: ((GeV*p/self.k3_R)**(1/self.GA3_R) + (p*GeV/(self.GA3_R-1)))/GeV
        Polytrope4_Read = lambda p: ((GeV*p/self.k4_R)**(1/self.GA4_R) + (p*GeV/(self.GA4_R-1)))/GeV
        function_list = [Polytrope1_Read, Polytrope2_Read, Polytrope3_Read, 
                         Polytrope4_Read, Polytrope1 , Polytrope2 , Polytrope3]
        
        d = (np.piecewise(p, condition, function_list))
        return (d)

    def R0(self,k,GA):
            dc = (mbar*(self.Pc*(GeV)/k)**(1/GA) + (self.Pc*(GeV)/(GA - 1)))/(GeV)
            P = self.Pc - dp 
            d = (mbar*(P*GeV/k)**(1/GA) +(P*GeV/(GA - 1)))/GeV
            nc = (((self.Pc*GeV)/k)**(1/GA))/GeV
            n = (((P*GeV)/k)**(1/GA))/GeV
            return  (((3/(2*pi*(3*self.Pc + dc)))*((math.log((dc + self.Pc)*n) - ((math.log((d + P)*nc)))))))**(0.5)

    def M0(self,k,GA):
            dc = (mbar*(self.Pc*(GeV)/k)**(1/GA) + (self.Pc*(GeV)/(GA - 1)))/(GeV)
            P = self.Pc - dp
            d = (mbar*(P*GeV/k)**(1/GA) +(P*GeV/(GA - 1)))/GeV
            nc = (((self.Pc*GeV)/k)**(1/GA))/GeV
            n = (((P*GeV)/k)**(1/GA))/GeV
            return ((((((math.log((dc + self.Pc)*n) - ((math.log((d + P)*nc)))))*(2*dc/(3*self.Pc + dc))))))*self.R0(k,GA)
    
        
    def f(self,x,P):
        m,r = x
        d = self.piecewise_polytrope_SFHx(P)
        dmdp = -(2*K1*(r**3)*d*(r-m))/((d + P)*(m + K1*P*(r**3)))
        drdp = -(2*r*(r - m))/((d + P)*(m + K1*P*(r**3)))
        return [dmdp , drdp]  
            
            
    def compute_MR(self):
        k2 = self.p1/((self.p1/self.k1)**(1/self.GA1) + self.p1/(self.GA1-1) - self.p1/(self.GA2-1))**self.GA2
        k3 = self.p2/((self.p2/k2)**(1/self.GA2) + self.p2/(self.GA2-1) - self.p2/(self.GA3-1))**self.GA3
       
        Ro =  self.R0(k = k3, GA = self.GA3)
        Mo =  self.M0(k = k3, GA = self.GA3)

        R = []
        M = []
        x_i = [Mo,Ro]
                         
        P0 = self.Pc - dp
            
        P1 = np.linspace(1e-6, P0, 1000)
        solve = odeint(self.f,x_i, P1[::-1])
                          
        m = solve[:,0]
        r = solve[:,1]

        M = (m[-2])
        R = (r[-2])*(Rs/km)
        return "M:",M ,"R:",R

In [176]:
A = TOV_Solver(Pc = 227.99999, 
               p1 = 0.053955053, 
               p2 = 0.797522989, 
               k1 = 1.8426632601064867, 
               GA1 = 3.88632843, 
               GA2 = 2.71342162,
               GA3 = 1.53107396,
               GA1_R = 1.58425, 
               GA2_R = 1.28733, 
               GA3_R = 0.62223,
               GA4_R = 1.35692, 
               k1_R = 2.8268581285298255e-16,
               k2_R = 5.063768639559009e-19,
               k3_R = 3.5610797610513604e-25 ,
               k4_R = 2.2297511225728095e-18, 
               P1_Read= 2.1445291264915162e-33,
               P2_Read= 5.332626487963225e-28,
               P3_Read = 1.7762371605771977e-27)


A.compute_MR()

/tmp/ipykernel_5144/2366035154.py:70: RuntimeWarning: invalid value encountered in power
  Polytrope1_Read = lambda p: ((GeV*p/self.k1_R)**(1/self.GA1_R) + (p*GeV/(self.GA1_R-1)))/GeV


('M:', 1.602295675679106, 'R:', 11.696637600843621)

In [185]:
polytrope1 = lambda k,d,GA: k*(d)**GA

P1_Read = polytrope1(k = 6.80110*10**(-9), d = 2.44034*10**7, GA = 1.58425)
P2_Read = polytrope1(k = 5.32697*10**1 , d = 3.78358*10**11, GA = 0.62233)
P3_Read = polytrope1(k = 3.99874*10**-8, d = 2.62780*10**12, GA = 1.35692)

In [186]:
P1_Read*0.624*10**-33/GeV

2.1445291264915162e-33

In [187]:
P2_Read*0.624*10**-33/GeV

5.332626487963225e-28

In [188]:
P3_Read*0.624*10**-33/GeV

1.7762371605771977e-27

In [189]:
GA1_R = 1.58425 
GA2_R = 1.28733 
GA3_R = 0.62223
GA4_R = 1.35692 

In [182]:
k1_R = ((1.7827*10**12)**GA1_R/(1.6022*10**33))*(1000)**(1-GA1_R)
k1_R

2.8268581285298255e-16

In [183]:
k2_R = (((1.7827*10**12)**GA2_R)/((1.6022*10**33))*(1000)**(1-GA2_R))
k2_R

5.063768639559009e-19

In [184]:
k3_R = (((1.7827*10**12)**GA3_R)/((1.6022*10**33))*(1000)**(1-GA3_R))
k3_R

3.5610797610513604e-25

In [146]:
k4_R = (((1.7827*10**12)**GA4_R)/((1.6022*10**33))*(1000)**(1-GA4_R))
k4_R

2.2297511225728095e-18